In [22]:
%matplotlib qt4
from __future__ import division
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("ticks", {"legend.frameon": True})

### Learning Curve

In [69]:
points = np.arange(0, 10, 0.001)
func = lambda x: -100/(x + 1)**0.8 + 100

plt.figure(num=None, figsize=(4, 3), dpi=160)
frame, = plt.plot(points, [func(p) for p in points])
plt.xlabel('time')
plt.ylabel('% of savings in relearning')
frame.axes.get_xaxis().set_ticks([])
sns.despine()
plt.tight_layout()

### Forgetting Curve

In [65]:
points = np.arange(0, 10, 0.001)
func_forget = lambda x: 100/(x + 1)**0.8

plt.figure(num=None, figsize=(4, 3), dpi=160)
frame, = plt.plot(points, [func_forget(p) for p in points])
plt.xlabel('time')
plt.ylabel('% of savings in relearning')
frame.axes.get_xaxis().set_ticks([])
sns.despine()
plt.tight_layout()